In [2]:
import os
import gym
import numpy as np
import cv2
from gym.spaces import Box
from nes_py.wrappers import JoypadSpace
import gym_super_mario_bros
from gym_super_mario_bros.actions import SIMPLE_MOVEMENT
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv, VecFrameStack
from stable_baselines3.common.callbacks import CheckpointCallback
from stable_baselines3.common.utils import set_random_seed

# 1. CARTELLE
CHECKPOINT_DIR = './train/nmod'
LOG_DIR = './logs/'
if not os.path.exists(CHECKPOINT_DIR): os.makedirs(CHECKPOINT_DIR)
if not os.path.exists(LOG_DIR): os.makedirs(LOG_DIR)

class MarioModernized(gym.Env):
    def __init__(self):
        super().__init__()
        inner_env = gym_super_mario_bros.make('SuperMarioBros-1-1-v0')
        self.mario = JoypadSpace(inner_env, SIMPLE_MOVEMENT)
        self.observation_space = Box(low=0, high=255, shape=(84, 84, 1), dtype=np.uint8)
        self.action_space = self.mario.action_space
        
        # Variabili di controllo
        self.max_x = 0
        self.stagnant_steps = 0
        self.skip_frames = 4 

    def _process_frame(self, frame):
        if frame is not None:
            gray = cv2.cvtColor(frame, cv2.COLOR_RGB2GRAY)
            resized = cv2.resize(gray, (84, 84), interpolation=cv2.INTER_AREA)
            return resized[:, :, np.newaxis]
        return np.zeros((84, 84, 1), dtype=np.uint8)

    def reset(self, seed=None, options=None):
        self.max_x = 0
        self.stagnant_steps = 0
        obs = self.mario.reset()
        self.mario.render()
        return self._process_frame(obs)

    def step(self, action):
        total_reward = 0
        for _ in range(self.skip_frames):
            obs, reward, done, info = self.mario.step(action)
            total_reward += reward
            if done: break

        self.mario.render()
        
        # --- NUOVA LOGICA PREMI (REWARD SHAPING) ---
        x_pos = info['x_pos']
        
        # 1. PREMIO PROGRESSIVO: Più è a destra, più riceve un piccolo bonus costante
        # Questo spinge Mario a non fermarsi mai. 
        # Ad esempio a x=1000 riceve +0.1 ogni step, a x=3000 riceve +0.3
        total_reward += (x_pos / 1000.0)

        # 2. CONTROLLO AVANZAMENTO
        if x_pos > self.max_x:
            self.max_x = x_pos
            self.stagnant_steps = 0
        else:
            self.stagnant_steps += 1

        # 3. PUNIZIONE SE STA FERMO
        if self.stagnant_steps > 100:
            total_reward -= 1.0 # Punizione per pigrizia
            
        # 4. BONUS BANDIERA (IL PREMIO ENORME)
        if info['flag_get']:
            print("!!! BANDIERA RAGGIUNTA !!!")
            total_reward += 500.0 # Un premio gigantesco
            done = True # Terminiamo il livello con successo
            
        # 5. CHIUSURA FORZATA SE PERSISTE A SBATTERE
        if self.stagnant_steps > 250:
            total_reward -= 5.0
            done = True

        obs = self._process_frame(obs)
        return obs, float(total_reward), done, info

if __name__ == "__main__":
    # SEED 42 per riproducibilità
    SEED = 42
    set_random_seed(SEED)
    
    env = DummyVecEnv([lambda: MarioModernized()])
    env.seed(SEED)
    env = VecFrameStack(env, n_stack=4, channels_order='last')

    checkpoint_callback = CheckpointCallback(
        save_freq=100000, 
        save_path=CHECKPOINT_DIR,
        name_prefix='mario_model'
    )

    model = PPO(
        "CnnPolicy", 
        env, 
        verbose=1, 
        learning_rate=0.0001,
        n_steps=512,
        batch_size=64,
        ent_coef=0.01, # Curiosità alta
        tensorboard_log=LOG_DIR,
        seed=SEED
    )

    print("Training con Incentivazione Progressiva e Bonus Bandiera attivo.")
    try:
        model.learn(total_timesteps=1000000, callback=checkpoint_callback)
    except KeyboardInterrupt:
        model.save("mario_model_final")

Using cuda device
Wrapping the env in a VecTransposeImage.
Training con Incentivazione Progressiva e Bonus Bandiera attivo.
Logging to ./logs/PPO_6


C:\Users\matte\Desktop\marioia\mario_stabile\Lib\site-packages\gym_super_mario_bros\smb_env.py:148: RuntimeWarning: overflow encountered in scalar subtract
  return (self.ram[0x86] - self.ram[0x071c]) % 256


----------------------------
| time/              |     |
|    fps             | 59  |
|    iterations      | 1   |
|    time_elapsed    | 8   |
|    total_timesteps | 512 |
----------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 70          |
|    iterations           | 2           |
|    time_elapsed         | 14          |
|    total_timesteps      | 1024        |
| train/                  |             |
|    approx_kl            | 0.007887229 |
|    clip_fraction        | 0.128       |
|    clip_range           | 0.2         |
|    entropy_loss         | -1.94       |
|    explained_variance   | 0.000756    |
|    learning_rate        | 0.0001      |
|    loss                 | 1.13e+03    |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.00518    |
|    value_loss           | 2.54e+03    |
-----------------------------------------
----------------------------------------
